In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
from ipywidgets import interactive
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Reading CSVs
~ ecs_updated (becomes ecs) - includes funding details for all 50 states + DC (needs to be merged on State)

~ enrollment_changes (becomes enrollments) - includes calculations for annual enrollment changes by NCES ID (filtered to local districts not affiliated with a supervisory union only and only includes columns needed for the loop or for merging, can be merged on State with ECS data or on NCES ID for other sources)

~ localdistricts (becomes localdistricts) - does not include enrollment change calculations, but does still include columns for locale, ZIP code, FTE count, pupil/teacher ratio, and year (can be merged on NCES ID or State depending on the need, shape aligns exactly to enrollment_changes)

~ enrollmentsdf (becomes fullnces) - primary dataframe until the loop; contains all columns and rows so it may ultimately prove to need more cleaning and restructuring than is helpful (can be merged by state or NCES ID)

In [2]:
ecs = pd.read_csv("../data/ecs_updated.csv")
ecs.head(3)

,Unnamed: 0,State,Primary Funding Model,Model Name,Notes,Base Amount (Y/N),Base Amount,Base Amount Legal Source,Student Count Method,Student Count Method Source,Special Education Funding Mechanism,Special Education Program Name,Special Education Amount (Dollar amount or weight),Is there a Cap or Minimum threshold in place?,Special Education Funding Description,Special Education Funding Source,English Learner Funding? (Y/N),English Learner Funding Mechanism,English Learner Program Name,English Learner Amount (Dollar amount or weight),ELL Amount Source,At-Risk Funding for Low-Income Students (Y/N),Low-Income Students Funding Mechanism,Low-Income Students Program Name,Low-Income Students Identifier,Low-Income Students Amount (Dollar amount or weight),Low-Income Funding Source,Gifted and Talented Funding? (Y/N),Gifted and Talented Funding Mechanism,Gifted and Talented Program Name,Gifted and Talented Amount (Dollar amount or weight),Gifted Funding Source,Small Size or Isolated Adjustment? (Y/N),Small Size Funding Mechanism,Small Size Program Name,Small Size Amount (Dollar amount or weight),Small Size Identifier,Small Size Identifier Source,Census Region,2020 Election Result,2020 Battleground State
0,0,ALABAMA,Resource-based allocation,Foundation Program,NaN,No,NaN,NaN,Membership average,Ala. Code § 16-13-232,Hybrid - census-based and high-cost services f...,Foundation Program and Catastrophic Trust Fund...,Census-based funding - the adjustment for spec...,"Yes, census-based cap of 5% ADM.",The Foundation Program adjusts ADM assuming 5%...,Ala. Admin. Code 290-2-1-.01 Ala. Code § 16-39-30,Yes,Categorical grant,English Language Learners Program,"Grant of $14,155,334 (2021-2022) to districts",Act 2021-342 (2021-2022 enacted budget),Yes,Categorical grant,At-Risk Student Program,Multiple - reduced or free - National School L...,"Provides $19,517,734 (2021-2022) for the program.",Ala. Admin. Code 290-2-1-.05 SB 189 (Budget bi...,Yes,Categorical grant,Gifted Students Program,"Total state appropriation of $4,825,000 FY2022",Ala. Code § 16-1-51 - amount in FY 2022 budget,No,NaN,NaN,NaN,NaN,NaN,South,Republican,No
1,1,ALASKA,Student-based foundation,State Aid to Public Schools,NaN,Yes,$5930 (2021-22),Alaska Stat. Ann. § 14.17.470,Enrollment count period,AK ST § 14.17.60,Multiple student weights,State Aid to Public Schools,An additional weight of 0.2 is applied to all ...,No,Districts receive a flat funding factor of 0.2...,"AK ST § 14.17.410, § 14.17.420",Yes,Census-based,State Aid to Public Schools,An additional weight of 0.2 is applied to each...,"AK ST § 14.17.410, § 14.17.420",No,NaN,NaN,NaN,NaN,NaN,Yes,Census-based,State Aid to Public Schools,0.2 funding factor is applied to each district...,"AK ST § 14.17.410, § 14.17.420",Yes,Multiple Student Weights,School size factor,"Based on their size, schools receive a funding...",Number of students,AK ST § 14.17.450,West,Republican,No
2,2,ARIZONA,Student-based foundation,Base Support Level,NaN,Yes,$4305.73 (2020-21) $4390.65 (2021-22),Ariz. Rev. Stat. Ann. § 15-901,Membership average,A.R.S. § 15-901,Hybrid - census-based and multiple student wei...,Base Support Level,Grade weights (combined funding for special ed...,No,Districts receive a flat funding factor of 0.1...,A.R.S. § 15-901 A.R.S. § 15-943,Yes,Hybrid - census-based and flat weight,Base Support Level,Grade weights: 0.158 funding factor is applied...,A.R.S. § 15-901 A.R.S. § 15-943,No,NaN,NaN,NaN,NaN,NaN,Yes,Hybrid - census-based and flat weight,Base Support level,Grade weights: 0.158 funding factor is applied...,A.R.S. § 15-901 A.R.S. § 15-943,Yes,Multiple Student Weights,Base Support Level,"Based on size, grade levels, and category (sma...",Multiple - Number of students and distance,AZ ST § 15-943 AZ ST § 15-901,West,Democrat,Yes


In [23]:
enrollments = pd.read_csv("../data/enrollment_changes.csv")
enrollments.head(3)

,Unnamed: 0,Agency Name,State Name,NCES ID,Student Count,Year,Enrollment Change
0,0,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2013,NaN
1,1,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2014,NaN
2,2,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2015,NaN


In [14]:
localdistricts = pd.read_csv("../data/localdistricts.csv")
localdistricts.head(3)

,Agency me,State me,State Abbreviation,NCES ID,District,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,Student Count,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,MONTGOMERY COUNTY,7,36057,1,21-Suburb: Large,1-Open,NaN,NaN,0.0,NaN,0.0,2013
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,JEFFERSON COUNTY,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2014
2,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,Jefferson County,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2015


In [16]:
#unlikely to use, but here for reference as needed
fullnces = pd.read_csv("../data/enrollmentsdf.csv")
fullnces.head(3)

,Agency me,State me,State Abbreviation,NCES ID,District,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,Student Count,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,MONTGOMERY COUNTY,7.0,36057.0,1,21-Suburb: Large,1-Open,NaN,NaN,0.0,NaN,0.0,2013
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,JEFFERSON COUNTY,6.0,36057.0,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2014
2,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,Jefferson County,6.0,36057.0,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2015


In [29]:
netchangedist = enrollments.groupby(['State Name', 'Agency Name', 'NCES ID'])['Enrollment Change'].sum().reset_index()
netchangedist.head()

,State Name,Agency Name,NCES ID,Enrollment Change
0,ALABAMA,AL INST DEAF AND BLIND,100009,0.0
1,ALABAMA,ALABAMA SCHOOL OF FINE ARTS,100018,0.0
2,ALABAMA,ALABAMA YOUTH SERVICES,100002,0.0
3,ALABAMA,ALABASTER CITY,100190,64.0
4,ALABAMA,ALBERTVILLE CITY,100005,1129.0


## Cleaning & Standardizing DataFrames

In [17]:
ecs = ecs.drop("Unnamed: 0", axis=1)
ecs.head(2)

,State,Primary Funding Model,Model Name,Notes,Base Amount (Y/N),Base Amount,Base Amount Legal Source,Student Count Method,Student Count Method Source,Special Education Funding Mechanism,Special Education Program Name,Special Education Amount (Dollar amount or weight),Is there a Cap or Minimum threshold in place?,Special Education Funding Description,Special Education Funding Source,English Learner Funding? (Y/N),English Learner Funding Mechanism,English Learner Program Name,English Learner Amount (Dollar amount or weight),ELL Amount Source,At-Risk Funding for Low-Income Students (Y/N),Low-Income Students Funding Mechanism,Low-Income Students Program Name,Low-Income Students Identifier,Low-Income Students Amount (Dollar amount or weight),Low-Income Funding Source,Gifted and Talented Funding? (Y/N),Gifted and Talented Funding Mechanism,Gifted and Talented Program Name,Gifted and Talented Amount (Dollar amount or weight),Gifted Funding Source,Small Size or Isolated Adjustment? (Y/N),Small Size Funding Mechanism,Small Size Program Name,Small Size Amount (Dollar amount or weight),Small Size Identifier,Small Size Identifier Source,Census Region,2020 Election Result,2020 Battleground State
0,ALABAMA,Resource-based allocation,Foundation Program,NaN,No,NaN,NaN,Membership average,Ala. Code § 16-13-232,Hybrid - census-based and high-cost services f...,Foundation Program and Catastrophic Trust Fund...,Census-based funding - the adjustment for spec...,"Yes, census-based cap of 5% ADM.",The Foundation Program adjusts ADM assuming 5%...,Ala. Admin. Code 290-2-1-.01 Ala. Code § 16-39-30,Yes,Categorical grant,English Language Learners Program,"Grant of $14,155,334 (2021-2022) to districts",Act 2021-342 (2021-2022 enacted budget),Yes,Categorical grant,At-Risk Student Program,Multiple - reduced or free - National School L...,"Provides $19,517,734 (2021-2022) for the program.",Ala. Admin. Code 290-2-1-.05 SB 189 (Budget bi...,Yes,Categorical grant,Gifted Students Program,"Total state appropriation of $4,825,000 FY2022",Ala. Code § 16-1-51 - amount in FY 2022 budget,No,NaN,NaN,NaN,NaN,NaN,South,Republican,No
1,ALASKA,Student-based foundation,State Aid to Public Schools,NaN,Yes,$5930 (2021-22),Alaska Stat. Ann. § 14.17.470,Enrollment count period,AK ST § 14.17.60,Multiple student weights,State Aid to Public Schools,An additional weight of 0.2 is applied to all ...,No,Districts receive a flat funding factor of 0.2...,"AK ST § 14.17.410, § 14.17.420",Yes,Census-based,State Aid to Public Schools,An additional weight of 0.2 is applied to each...,"AK ST § 14.17.410, § 14.17.420",No,NaN,NaN,NaN,NaN,NaN,Yes,Census-based,State Aid to Public Schools,0.2 funding factor is applied to each district...,"AK ST § 14.17.410, § 14.17.420",Yes,Multiple Student Weights,School size factor,"Based on their size, schools receive a funding...",Number of students,AK ST § 14.17.450,West,Republican,No


In [18]:
ecs.rename(columns={'State':'State Name'}, inplace=True)
ecs.head(1)

,State Name,Primary Funding Model,Model Name,Notes,Base Amount (Y/N),Base Amount,Base Amount Legal Source,Student Count Method,Student Count Method Source,Special Education Funding Mechanism,Special Education Program Name,Special Education Amount (Dollar amount or weight),Is there a Cap or Minimum threshold in place?,Special Education Funding Description,Special Education Funding Source,English Learner Funding? (Y/N),English Learner Funding Mechanism,English Learner Program Name,English Learner Amount (Dollar amount or weight),ELL Amount Source,At-Risk Funding for Low-Income Students (Y/N),Low-Income Students Funding Mechanism,Low-Income Students Program Name,Low-Income Students Identifier,Low-Income Students Amount (Dollar amount or weight),Low-Income Funding Source,Gifted and Talented Funding? (Y/N),Gifted and Talented Funding Mechanism,Gifted and Talented Program Name,Gifted and Talented Amount (Dollar amount or weight),Gifted Funding Source,Small Size or Isolated Adjustment? (Y/N),Small Size Funding Mechanism,Small Size Program Name,Small Size Amount (Dollar amount or weight),Small Size Identifier,Small Size Identifier Source,Census Region,2020 Election Result,2020 Battleground State
0,ALABAMA,Resource-based allocation,Foundation Program,NaN,No,NaN,NaN,Membership average,Ala. Code § 16-13-232,Hybrid - census-based and high-cost services f...,Foundation Program and Catastrophic Trust Fund...,Census-based funding - the adjustment for spec...,"Yes, census-based cap of 5% ADM.",The Foundation Program adjusts ADM assuming 5%...,Ala. Admin. Code 290-2-1-.01 Ala. Code § 16-39-30,Yes,Categorical grant,English Language Learners Program,"Grant of $14,155,334 (2021-2022) to districts",Act 2021-342 (2021-2022 enacted budget),Yes,Categorical grant,At-Risk Student Program,Multiple - reduced or free - National School L...,"Provides $19,517,734 (2021-2022) for the program.",Ala. Admin. Code 290-2-1-.05 SB 189 (Budget bi...,Yes,Categorical grant,Gifted Students Program,"Total state appropriation of $4,825,000 FY2022",Ala. Code § 16-1-51 - amount in FY 2022 budget,No,NaN,NaN,NaN,NaN,NaN,South,Republican,No


In [19]:
enrollments = enrollments.drop("Unnamed: 0", axis=1)
enrollments.head(2)

,Agency Name,State Name,NCES ID,Student Count,Year,Enrollment Change,Net Enrollment Change
0,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2013,NaN,NaN
1,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2014,NaN,NaN


In [26]:
netchangedist.rename(columns={'Enrollment Change':'Net Enrollment Change'}, inplace=True)
netchangedist.head(2)

,State Name,Agency Name,Net Enrollment Change
0,ALABAMA,AL INST DEAF AND BLIND,0.0
1,ALABAMA,ALABAMA SCHOOL OF FINE ARTS,0.0


## Merging DataFrames
Will also include dropping duplicate/unnecessary columns, standardizing column names, and reordering columns for efficiency.

In [33]:
netenrollments = enrollments.merge(netchangedist, on='NCES ID', suffixes=('_nces', '_net'))
netenrollments.head(3)

,Unnamed: 0,Agency Name_nces,State Name_nces,NCES ID,Student Count,Year,Enrollment Change_nces,State Name_net,Agency Name_net,Enrollment Change_net
0,0,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2013,NaN,ALABAMA,ALABAMA YOUTH SERVICES,0.0
1,1,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2014,NaN,ALABAMA,ALABAMA YOUTH SERVICES,0.0
2,2,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2015,NaN,ALABAMA,ALABAMA YOUTH SERVICES,0.0


In [34]:
netenrollments = netenrollments.drop(['Unnamed: 0', 'State Name_net', 'Agency Name_net'], axis=1)
netenrollments.head(3)

,Agency Name_nces,State Name_nces,NCES ID,Student Count,Year,Enrollment Change_nces,Enrollment Change_net
0,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2013,NaN,0.0
1,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2014,NaN,0.0
2,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2015,NaN,0.0


In [35]:
netenrollments.rename(columns={'Agency Name_nces':'Agency Name', 'State Name_nces':'State Name', 'Enrollment Change_nces':'Annual Enrollment Change', 'Enrollment Change_net':'Net Enrollment Change'}, inplace=True)
netenrollments.head(3)

,Agency Name,State Name,NCES ID,Student Count,Year,Annual Enrollment Change,Net Enrollment Change
0,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2013,NaN,0.0
1,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2014,NaN,0.0
2,ALABAMA YOUTH SERVICES,ALABAMA,100002,NaN,2015,NaN,0.0


In [36]:
enrollments.shape

(104922, 7)

In [52]:
netchangedist.shape

(13464, 4)

In [37]:
netenrollments.shape

(104922, 7)

In [38]:
localdistricts.shape

(104922, 17)

In [48]:
netandlocal = localdistricts.merge(netenrollments, on=['NCES ID', 'Year'], how='left', suffixes=('_loc', '_net'))
netandlocal.shape
#dataframes were the same shape - needed left merge rather than outer to avoid 8x duplicates

(104922, 22)

In [49]:
netandlocal.head(3)

,Agency me,State me,State Abbreviation,NCES ID,District,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,Student Count_loc,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year,Agency Name,State Name,Student Count_net,Annual Enrollment Change,Net Enrollment Change
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,MONTGOMERY COUNTY,7,36057,1,21-Suburb: Large,1-Open,NaN,NaN,0.0,NaN,0.0,2013,ALABAMA YOUTH SERVICES,ALABAMA,NaN,NaN,0.0
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,JEFFERSON COUNTY,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2014,ALABAMA YOUTH SERVICES,ALABAMA,NaN,NaN,0.0
2,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,ALABAMA YOUTH SERVICES,Jefferson County,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,NaN,2015,ALABAMA YOUTH SERVICES,ALABAMA,NaN,NaN,0.0


In [50]:
netandlocal = netandlocal.drop(['Student Count_loc', 'Agency Name', 'District', 'State Name'], axis=1)
netandlocal.head(3)

,Agency me,State me,State Abbreviation,NCES ID,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year,Student Count_net,Annual Enrollment Change,Net Enrollment Change
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,MONTGOMERY COUNTY,7,36057,1,21-Suburb: Large,1-Open,NaN,0.0,NaN,0.0,2013,NaN,NaN,0.0
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,JEFFERSON COUNTY,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,2014,NaN,NaN,0.0
2,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,Jefferson County,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,2015,NaN,NaN,0.0


In [51]:
netandlocal.rename(columns={'Agency me':'Agency Name', 'State me':'State Name', 'Student Count_net':'Student Count'}, inplace=True)
netandlocal.head(3)

,Agency Name,State Name,State Abbreviation,NCES ID,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year,Student Count,Annual Enrollment Change,Net Enrollment Change
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,MONTGOMERY COUNTY,7,36057,1,21-Suburb: Large,1-Open,NaN,0.0,NaN,0.0,2013,NaN,NaN,0.0
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,JEFFERSON COUNTY,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,2014,NaN,NaN,0.0
2,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002,Jefferson County,6,36057,1,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,2015,NaN,NaN,0.0


In [56]:
netandlocal.shape

(104922, 18)

In [63]:
netfunding = netandlocal.merge(ecs, on='State Name', how='outer', suffixes=('_net', '_ecs'))
netfunding.shape

(104929, 57)

In [64]:
netandlocal.info()
#all rows should have a state name given the df shape and non-null count

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104922 entries, 0 to 104921
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Agency Name               104922 non-null  object 
 1   State Name                104922 non-null  object 
 2   State Abbreviation        104922 non-null  object 
 3   NCES ID                   104922 non-null  int64  
 4   County                    104898 non-null  object 
 5   School Count              104922 non-null  int64  
 6   ZIP Code                  104922 non-null  int64  
 7   District Type             104922 non-null  int64  
 8   Locale                    104898 non-null  object 
 9   Start of Year Status      104922 non-null  object 
 10  Updated Status            91850 non-null   object 
 11  FTE Teachers              101295 non-null  float64
 12  Pupil/Teacher Ratio       99181 non-null   float64
 13  Total Staff               101016 non-null  f

In [58]:
#netfunding.to_csv("../data/netfunding.csv")

In [61]:
#netandlocal.to_csv("../data/netandlocal2.csv")

In [67]:
netfunding.head(2)

,Agency Name,State Name,State Abbreviation,NCES ID,County,School Count,ZIP Code,District Type,Locale,Start of Year Status,Updated Status,FTE Teachers,Pupil/Teacher Ratio,Total Staff,Year,Student Count,Annual Enrollment Change,Net Enrollment Change,Primary Funding Model,Model Name,Notes,Base Amount (Y/N),Base Amount,Base Amount Legal Source,Student Count Method,Student Count Method Source,Special Education Funding Mechanism,Special Education Program Name,Special Education Amount (Dollar amount or weight),Is there a Cap or Minimum threshold in place?,Special Education Funding Description,Special Education Funding Source,English Learner Funding? (Y/N),English Learner Funding Mechanism,English Learner Program Name,English Learner Amount (Dollar amount or weight),ELL Amount Source,At-Risk Funding for Low-Income Students (Y/N),Low-Income Students Funding Mechanism,Low-Income Students Program Name,Low-Income Students Identifier,Low-Income Students Amount (Dollar amount or weight),Low-Income Funding Source,Gifted and Talented Funding? (Y/N),Gifted and Talented Funding Mechanism,Gifted and Talented Program Name,Gifted and Talented Amount (Dollar amount or weight),Gifted Funding Source,Small Size or Isolated Adjustment? (Y/N),Small Size Funding Mechanism,Small Size Program Name,Small Size Amount (Dollar amount or weight),Small Size Identifier,Small Size Identifier Source,Census Region,2020 Election Result,2020 Battleground State
0,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002.0,MONTGOMERY COUNTY,7.0,36057.0,1.0,21-Suburb: Large,1-Open,NaN,0.0,NaN,0.0,2013.0,NaN,NaN,0.0,Resource-based allocation,Foundation Program,NaN,No,NaN,NaN,Membership average,Ala. Code § 16-13-232,Hybrid - census-based and high-cost services f...,Foundation Program and Catastrophic Trust Fund...,Census-based funding - the adjustment for spec...,"Yes, census-based cap of 5% ADM.",The Foundation Program adjusts ADM assuming 5%...,Ala. Admin. Code 290-2-1-.01 Ala. Code § 16-39-30,Yes,Categorical grant,English Language Learners Program,"Grant of $14,155,334 (2021-2022) to districts",Act 2021-342 (2021-2022 enacted budget),Yes,Categorical grant,At-Risk Student Program,Multiple - reduced or free - National School L...,"Provides $19,517,734 (2021-2022) for the program.",Ala. Admin. Code 290-2-1-.05 SB 189 (Budget bi...,Yes,Categorical grant,Gifted Students Program,"Total state appropriation of $4,825,000 FY2022",Ala. Code § 16-1-51 - amount in FY 2022 budget,No,NaN,NaN,NaN,NaN,NaN,South,Republican,No
1,ALABAMA YOUTH SERVICES,ALABAMA,AL,100002.0,JEFFERSON COUNTY,6.0,36057.0,1.0,41-Rural: Fringe,1-Open,1-Open,NaN,NaN,NaN,2014.0,NaN,NaN,0.0,Resource-based allocation,Foundation Program,NaN,No,NaN,NaN,Membership average,Ala. Code § 16-13-232,Hybrid - census-based and high-cost services f...,Foundation Program and Catastrophic Trust Fund...,Census-based funding - the adjustment for spec...,"Yes, census-based cap of 5% ADM.",The Foundation Program adjusts ADM assuming 5%...,Ala. Admin. Code 290-2-1-.01 Ala. Code § 16-39-30,Yes,Categorical grant,English Language Learners Program,"Grant of $14,155,334 (2021-2022) to districts",Act 2021-342 (2021-2022 enacted budget),Yes,Categorical grant,At-Risk Student Program,Multiple - reduced or free - National School L...,"Provides $19,517,734 (2021-2022) for the program.",Ala. Admin. Code 290-2-1-.05 SB 189 (Budget bi...,Yes,Categorical grant,Gifted Students Program,"Total state appropriation of $4,825,000 FY2022",Ala. Code § 16-1-51 - amount in FY 2022 budget,No,NaN,NaN,NaN,NaN,NaN,South,Republican,No
